# 📝 Summarizer Agent 

This notebook demonstrates an **LLM-powered text summarizer agent** with advanced features:
- Supports multiple summary modes: paragraph, bullet points, key insights, TL;DR, structured JSON
- Adjustable length (short, medium, long)
- Custom tone: neutral, formal, casual, professional, persuasive
- Works with **raw text, URLs, and PDFs**
- Option to keep short quotes from the text
- Interactive demo with dropdowns and text input

This project shows how to build **intelligent summarization agents** using LLMs.

## ⚙️ Setup

We will install the required Python libraries for summarization, text extraction, and interactive widgets.

In [ ]:
!pip install openai requests PyPDF2 beautifulsoup4 ipywidgets

## 🧠 Summarizer Agent Class

The `LLMSummarizerAgent` wraps the OpenAI LLM into a convenient interface.

It provides methods for:
- `summarize()` → generate summaries with tone & style
- `summarize_url()` → fetch webpage text and summarize
- `summarize_pdf()` → read a PDF and summarize
- `multi_summary()` → produce different summary formats at once

In [ ]:
import requests
from bs4 import BeautifulSoup
import PyPDF2
from openai import OpenAI

class LLMSummarizerAgent:
    def __init__(self, model="gpt-4o-mini"):
        self.model = model
        self.client = OpenAI()

    def summarize(self, text: str, length: str = "short", mode: str = "paragraph", tone: str = "neutral", keep_quotes: bool = False) -> str:
        instructions = {
            "paragraph": "Summarize into a clear, single paragraph.",
            "bullets": "Summarize into concise bullet points.",
            "key-insights": "Summarize into 3-5 key insights.",
            "tldr": "Summarize into a one-line TL;DR.",
            "json": "Summarize into structured JSON with sections: main_points, conclusion."
        }
        prompt = f"""
        Summarize the following text in a {length} form.
        Style: {instructions.get(mode, 'paragraph')}.
        Tone: {tone}.
        { 'Preserve short important quotes where useful.' if keep_quotes else '' }

        Text:
        {text}
        """

        resp = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4
        )
        return resp.choices[0].message.content.strip()

    def summarize_url(self, url: str, **kwargs) -> str:
        html = requests.get(url).text
        soup = BeautifulSoup(html, "html.parser")
        text = " ".join(p.get_text() for p in soup.find_all("p"))
        return self.summarize(text, **kwargs)

    def summarize_pdf(self, file_path: str, **kwargs) -> str:
        with open(file_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            text = " ".join(page.extract_text() or "" for page in reader.pages)
        return self.summarize(text, **kwargs)

    def multi_summary(self, text: str, modes: list, **kwargs) -> dict:
        results = {}
        for mode in modes:
            results[mode] = self.summarize(text, mode=mode, **kwargs)
        return results

## 📝 Summarization Prompt (used internally)

We guide the LLM with clear instructions for consistent, context-aware summarization:

```
You are an advanced summarization assistant.
Your goals:
1. Produce summaries that are clear, concise, and faithful to the original text.
2. Respect the chosen length (short, medium, long).
3. Adapt to the requested mode (paragraph, bullets, insights, TL;DR, JSON).
4. Maintain the requested tone (neutral, formal, casual, professional, persuasive).
5. Optionally preserve important short quotes.
```

## 🎛️ Interactive Summarization Demo

Use the widgets below to test summarization:
- Enter some text
- Select summary length & mode
- Choose a tone
- Optionally keep quotes
- Click **Summarize**

In [ ]:
import ipywidgets as widgets
from IPython.display import display

agent = LLMSummarizerAgent()

text_input = widgets.Textarea(value="Artificial Intelligence is transforming industries...", placeholder="Enter text to summarize...", description="Text:", layout=widgets.Layout(width="100%", height="150px"))
length = widgets.Dropdown(options=["short", "medium", "long"], value="short", description="Length:")
mode = widgets.Dropdown(options=["paragraph", "bullets", "key-insights", "tldr", "json"], value="paragraph", description="Mode:")
tone = widgets.Dropdown(options=["neutral", "formal", "casual", "professional", "persuasive"], value="neutral", description="Tone:")
keep_quotes = widgets.Checkbox(value=False, description="Keep quotes")

button = widgets.Button(description="Summarize", button_style="success")
output = widgets.Output()

def on_summarize_clicked(b):
    output.clear_output()
    with output:
        result = agent.summarize(text_input.value, length.value, mode.value, tone.value, keep_quotes.value)
        print("📝 Summary:", result)

button.on_click(on_summarize_clicked)
display(text_input, length, mode, tone, keep_quotes, button, output)

## 📊 Multi-Format Example

Here we generate summaries in **multiple formats at once** (e.g., paragraph + bullets + TL;DR).
This is useful for producing summaries for different audiences.

In [ ]:
agent = LLMSummarizerAgent()
results = agent.multi_summary("AI is reshaping industries, from healthcare to finance, by improving efficiency and enabling new capabilities.", ["paragraph", "bullets", "tldr"])
for mode, summary in results.items():
    print(f"{mode.upper()}:\n{summary}\n")

## 🚀 Next Steps

You can extend this agent by:
- Adding **topic extraction** to identify main themes
- Generating **executive summaries** for business reports
- Adding **abstractive vs extractive** summarization options
- Deploying as a **web app** with Gradio/Streamlit